In [ ]:
from IPython.display import display
import ipywidgets as ipw
import widgets
import utils
import os

In [ ]:
CONFIG = utils.read_json("config.json")
CONFIG_ELN = utils.get_aiidalab_eln_config()
# CONFIG_ELN = utils.read_json("eln_config.json")
OPENBIS_SESSION, SESSION_DATA = utils.connect_openbis(CONFIG_ELN["url"], CONFIG_ELN["token"])

project_selector = widgets.ProjectSelectionWidget()
project_selector.load_dropdown_box()

drafts_selector = widgets.ObjectMultipleSelectionWidget("Draft")
grants_selector = widgets.ObjectMultipleSelectionWidget("Grants")
grants_selector.load_selector("GRANT")
authors_selector = widgets.ObjectMultipleSelectionWidget("Authors")
authors_selector.load_selector("PERSON")

publication_widgets = widgets.ObjectPropertiesWidgets("Publication")

publication_support_files = ipw.FileUpload(multiple = True)

increase_buttons_size = utils.HTML(data = ''.join(CONFIG["save_home_buttons_settings"]))
create_button = utils.Button(
    description = '', disabled = False, button_style = '', tooltip = 'Save', 
    icon = 'save', layout = ipw.Layout(width = '100px', height = '50px')
)
quit_button = utils.Button(
    description = '', disabled = False, button_style = '', 
    tooltip = 'Main menu', icon = 'home', layout = ipw.Layout(width = '100px', height = '50px')
)
save_close_buttons_hbox = ipw.HBox([create_button, quit_button])

In [ ]:
def close_notebook(b):
    display(utils.Javascript(data = 'window.location.replace("home.ipynb")'))

def upload_datasets(method_object, support_files_widget):
    for filename in support_files_widget.value:
        file_info = support_files_widget.value[filename]
        utils.save_file(file_info['content'], filename)
        OPENBIS_SESSION.new_dataset(type = 'RAW_DATA', sample = method_object, files = [filename]).save()
        os.remove(filename)

def load_drafts(change):
    if project_selector.dropdown.value != -1:
        drafts_selector.load_selector("DRAFT", project_selector.dropdown.value)
    
def create_publication_openbis(b):
    publication_props = {}
    for prop in CONFIG["objects"]["Publication"]["properties"]:
        publication_props[prop] = publication_widgets.properties_widgets[prop].value
        
    publication_parents = []
    publication_parents.extend(list(authors_selector.selector.value))
    publication_parents.extend(list(drafts_selector.selector.value))
    publication_parents.extend(list(grants_selector.selector.value))
    publication_object = utils.create_openbis_object(
        OPENBIS_SESSION, type="PUBLICATION_CUSTOM", 
        collection="/PUBLICATIONS/PUBLIC_REPOSITORIES/PUBLICATIONS_COLLECTION", 
        props = publication_props, parents = publication_parents
    )
    upload_datasets(publication_object, publication_support_files)
    display(utils.Javascript(data = "alert('Upload successful!')"))

# Create publication

## Select project

In [ ]:
display(project_selector.dropdown_boxes)
project_selector.dropdown.observe(load_drafts, names = 'value')

## Select authors

In [ ]:
display(authors_selector)

## Select grants

In [ ]:
display(grants_selector)

## Select drafts

In [ ]:
display(drafts_selector)

## Properties

In [ ]:
display(publication_widgets)

## Support files

In [ ]:
display(publication_support_files)

## Save results

In [ ]:
display(save_close_buttons_hbox)
display(increase_buttons_size)
create_button.on_click(create_publication_openbis)
quit_button.on_click(close_notebook)